In [1]:
from scienceworld import ScienceWorldEnv
import re
from sources.agent.scienceworld import parse_observation

In [2]:
env = ScienceWorldEnv("", "", envStepLimit=100)

In [3]:
taskNames = env.getTaskNames()
print("Task Names: " + str(taskNames))

# Choose task
taskName = taskNames[0]  # Just get first task
env.load(taskName, 0, "")  # Load the task, so we have access to some extra accessors e.g. getRandomVariationTrain()
maxVariations = env.getMaxVariations(taskName)
taskName, maxVariations

Task Names: ['boil', 'change-the-state-of-matter-of', 'chemistry-mix', 'chemistry-mix-paint-secondary-color', 'chemistry-mix-paint-tertiary-color', 'find-animal', 'find-living-thing', 'find-non-living-thing', 'find-plant', 'freeze', 'grow-fruit', 'grow-plant', 'identify-life-stages-1', 'identify-life-stages-2', 'inclined-plane-determine-angle', 'inclined-plane-friction-named-surfaces', 'inclined-plane-friction-unnamed-surfaces', 'lifespan-longest-lived', 'lifespan-longest-lived-then-shortest-lived', 'lifespan-shortest-lived', 'measure-melting-point-known-substance', 'measure-melting-point-unknown-substance', 'melt', 'mendelian-genetics-known-plant', 'mendelian-genetics-unknown-plant', 'power-component', 'power-component-renewable-vs-nonrenewable-energy', 'test-conductivity', 'test-conductivity-of-unknown-substances', 'use-thermometer']


('boil', 30)

In [4]:
def parse(observation: str, inventory: str) -> list[str]:
    x = re.search(r"([\S\s]*?)(?:In it, you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", observation)
    groups = x.groups()

    location = groups[0]
    objects = groups[1]
    doors = groups[2]

    loc_split = [location.strip()]
    obs_split = [obs.strip() for obs in objects.split('\n') if len(obs.strip()) > 0]
    obs_split = [f"You see a {obs}" for obs in obs_split]
    doors_split = [door.strip() for door in doors.split('\n') if len(door.strip()) > 0]
    inventory_items = inventory.replace('\n', ' ').replace('\t', '')
    return loc_split + obs_split + doors_split + [inventory_items]

In [5]:
#randVariationIdx = env.getRandomVariationTest()
randVariationIdx = 0
env.load(taskName, randVariationIdx)
# Reset the environment
observation, info = env.reset()

print(f"Task Name: " + taskName + " variation " + str(randVariationIdx))
print("Task Variation: " + str(randVariationIdx) + " / " + str(maxVariations))
print("Task Description: " + str(env.getTaskDescription()))

score = 0.0
curIter = 0

# Run one episode until we reach a stopping condition (including exceeding the maximum steps)
action_str = "look around"  # First action
observation, reward, isCompleted, info = env.step(action_str)

Task Name: boil variation 0
Task Variation: 0 / 30
Task Description: Your task is to boil water. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


In [8]:
env.getValidActionObjectCombinationsWithTemplates()

[{'action': 'open art studio door',
  'template_id': 0,
  'obj_ids': [17580],
  'type_ids': [140]},
 {'action': 'open bedroom door',
  'template_id': 0,
  'obj_ids': [17568],
  'type_ids': [140]},
 {'action': 'open door to workshop',
  'template_id': 0,
  'obj_ids': [17574],
  'type_ids': [140]},
 {'action': 'open door to kitchen',
  'template_id': 0,
  'obj_ids': [17562],
  'type_ids': [140]},
 {'action': 'open door to living room',
  'template_id': 0,
  'obj_ids': [17565],
  'type_ids': [140]},
 {'action': 'open door to greenhouse',
  'template_id': 0,
  'obj_ids': [17577],
  'type_ids': [140]},
 {'action': 'look around', 'template_id': 3, 'obj_ids': [], 'type_ids': []},
 {'action': 'look at art studio door',
  'template_id': 4,
  'obj_ids': [17580],
  'type_ids': [140]},
 {'action': 'look at greenhouse',
  'template_id': 4,
  'obj_ids': [17418],
  'type_ids': [132]},
 {'action': 'look at bedroom door',
  'template_id': 4,
  'obj_ids': [17568],
  'type_ids': [140]},
 {'action': 'look

In [6]:
look = info['look']
print(look)
inv = info['inv']

beliefs_a = parse(look, inventory=inv)
beliefs_a

This room is called the hallway. In it, you see: 
	the agent
	a substance called air
	a picture
You also see:
	A door to the art studio (that is closed)
	A door to the bedroom (that is closed)
	A door to the greenhouse (that is closed)
	A door to the kitchen (that is closed)
	A door to the living room (that is closed)
	A door to the workshop (that is closed)



['This room is called the hallway.',
 'You see a the agent',
 'You see a a substance called air',
 'You see a a picture',
 'A door to the art studio (that is closed)',
 'A door to the bedroom (that is closed)',
 'A door to the greenhouse (that is closed)',
 'A door to the kitchen (that is closed)',
 'A door to the living room (that is closed)',
 'A door to the workshop (that is closed)',
 'In your inventory, you see: an orange ']

In [7]:
observation, reward, isCompleted, info = env.step("open door to the art studio")

In [8]:
beliefs_b = parse(info['look'], inventory=inv)
beliefs_b

['This room is called the hallway.',
 'You see a the agent',
 'You see a a substance called air',
 'You see a a picture',
 'A door to the art studio (that is open)',
 'A door to the bedroom (that is closed)',
 'A door to the greenhouse (that is closed)',
 'A door to the kitchen (that is closed)',
 'A door to the living room (that is closed)',
 'A door to the workshop (that is closed)',
 'In your inventory, you see: an orange ']

In [9]:
list(set(beliefs_a) - set(beliefs_b))

['A door to the art studio (that is closed)']

In [10]:
observation, reward, isCompleted, info = env.step("pick up picture")
beliefs_c = parse(info['look'], inventory=inv)
beliefs_c

['This room is called the hallway.',
 'You see a the agent',
 'You see a a substance called air',
 'A door to the art studio (that is open)',
 'A door to the bedroom (that is closed)',
 'A door to the greenhouse (that is closed)',
 'A door to the kitchen (that is closed)',
 'A door to the living room (that is closed)',
 'A door to the workshop (that is closed)',
 'In your inventory, you see: an orange ']

In [11]:
list(set(beliefs_b) - set(beliefs_c))

['You see a a picture']

In [12]:
# go action must have specific preconditions (go to the x -> the door to the x must be opened)
# they change drastically the belief
observation, reward, isCompleted, info = env.step("go to the art studio")
beliefs_d = parse(info['look'], inventory=inv)
beliefs_d

['This room is called the art studio.',
 'You see a the agent',
 'You see a a substance called air',
 'You see a a large cupboard. The large cupboard door is closed.',
 'You see a a table. On the table is: a jug (containing nothing).',
 'You see a a wood cup (containing blue paint)',
 'You see a a wood cup (containing yellow paint)',
 'You see a a wood cup (containing red paint)',
 'A door to the hallway (that is open)',
 'In your inventory, you see: an orange ']

In [68]:
list(set(beliefs_d) - set(beliefs_c))

['You see a a large cupboard. The large cupboard door is closed.',
 'You see a a wood cup (containing red paint)',
 'This room is called the art studio.',
 'You see a a table. On the table is: a jug (containing nothing).',
 'You see a a wood cup (containing yellow paint)',
 'You see a a wood cup (containing blue paint)',
 'A door to the hallway (that is open)']